In [7]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv")


display(x_train.head())
display(y_train.head())
display(x_test.head())
train = pd.merge(x_train, y_train, on='ID')
target= x_test.copy()

,ID,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,id
0,0,Female,Loyal Customer,54,Personal Travel,Eco,1068,3,4,3,...,5,5,3,5,3,5,3,47,22.0,NaN
1,2,Male,Loyal Customer,20,Personal Travel,Eco,1546,4,4,4,...,4,3,3,4,4,4,4,5,2.0,NaN
2,3,Male,Loyal Customer,59,Business travel,Business,2962,0,4,0,...,1,1,1,1,5,1,4,54,46.0,NaN
3,4,Male,Loyal Customer,35,Business travel,Eco Plus,106,5,4,4,...,5,2,1,5,4,4,5,130,121.0,NaN
4,5,Female,Loyal Customer,9,Business travel,Business,2917,3,3,3,...,4,4,4,5,4,3,4,0,0.0,NaN


,ID,satisfaction
0,0,neutral or dissatisfied
1,2,neutral or dissatisfied
2,3,satisfied
3,4,satisfied
4,5,satisfied


,ID,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,id
0,1,Male,Loyal Customer,28,Business travel,Business,3414,2,2,2,...,2,4,5,4,4,4,2,0,0.0,40819.0
1,16,Female,disloyal Customer,34,Business travel,Eco,2556,1,1,1,...,2,5,1,5,1,3,2,15,0.0,33335.0
2,17,Female,Loyal Customer,23,Personal Travel,Eco,363,2,5,2,...,5,3,2,4,5,4,5,0,0.0,20749.0
3,25,Male,Loyal Customer,55,Business travel,Business,2079,1,1,1,...,1,1,1,1,4,1,3,0,0.0,92194.0
4,27,Female,Loyal Customer,44,Business travel,Eco,192,2,4,5,...,1,1,2,1,5,1,1,23,13.0,13904.0


In [21]:
# print(train.shape)
# print()
# print(train.info())
# print()
# print(train.describe())
# print()
# print(train.isnull().sum())
# print(target.shape)
# print()
# print(target.info())
# print()
# print(target.describe())
# print()
# print(train.isnull().sum())
x = train.drop(['ID','id','satisfaction'], axis=1).fillna(0)
y = train['satisfaction']
target_x = target.drop(['ID','id'], axis=1).fillna(0)

obj_col = x.select_dtypes(include='object').columns
# for col in obj_col:
#   print(col)
#   print(x[col].unique())
#   print(target_x[col].unique())

x = pd.get_dummies(x, columns =obj_col, dtype=int)
target_x = pd.get_dummies(target_x, columns = obj_col, dtype=int)

from sklearn.model_selection import train_test_split
rs = 810
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=rs)

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

rf = RandomForestClassifier(random_state=rs)

param = {'n_estimators':range(15,20),
         'max_depth':range(15,20)}
gscv = GridSearchCV(rf, param_grid=param, cv=5)
gscv.fit(x_train, y_train)
print(gscv.best_params_)
model = gscv.best_estimator_

pred_val = model.predict(x_val)
pred_val_proba = model.predict_proba(x_val)[:,1]

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
print(f1_score(y_val, pred_val,pos_label = 'neutral or dissatisfied'))
print(accuracy_score(y_val, pred_val))
print(roc_auc_score(y_val, pred_val_proba))

pred = model.predict(target_x)
result = pd.DataFrame({'ID':target['ID'], 'pred':pred})
result.to_csv('result.csv', index=False)
print(result)

{'max_depth': 17, 'n_estimators': 19}
0.9660090573109156
0.9607218045112782
0.9925713119202929
           ID                     pred
0           1  neutral or dissatisfied
1          16  neutral or dissatisfied
2          17  neutral or dissatisfied
3          25  neutral or dissatisfied
4          27                satisfied
...       ...                      ...
20776  103895  neutral or dissatisfied
20777  103896  neutral or dissatisfied
20778  103897                satisfied
20779  103900  neutral or dissatisfied
20780  103902  neutral or dissatisfied

[20781 rows x 2 columns]


In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,55882,Male,Loyal Customer,42,Business travel,Business,473,1,1,3,...,5,5,5,5,3,5,3,3,0.0,satisfied
1,31906,Female,Loyal Customer,31,Business travel,Business,3271,4,4,4,...,3,3,5,5,3,4,3,1,5.0,satisfied
2,85481,Male,Loyal Customer,51,Personal Travel,Eco,214,5,0,5,...,3,4,4,4,4,4,3,2,0.0,satisfied
3,102134,Male,Loyal Customer,44,Business travel,Business,3097,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied
4,31580,Male,Loyal Customer,33,Business travel,Business,3286,1,1,1,...,4,2,5,2,5,1,4,0,0.0,satisfied


In [21]:
target = df[df['Arrival Delay in Minutes'].isnull()]
ans = target.groupby(['Class','satisfaction'],as_index=False).size().pivot(index='Class', columns='satisfaction')
print(ans.head(1))

                                size          
satisfaction neutral or dissatisfied satisfied
Class                                         
Business                          36        76


In [22]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.918150,214.186611,23823.492888,7.290878,341.173322,411.424483,19.585002,25.057375,4.028958,0
1,5.422446,205.266080,18542.957451,5.491963,306.702227,382.080129,10.504023,67.493450,2.911751,1
2,7.341547,187.672402,21273.457066,7.784003,NaN,332.084293,16.842334,55.019151,4.025644,0
3,9.056245,197.666301,17403.532167,7.688917,337.460176,414.766631,15.349869,63.696746,3.319354,0
4,5.039374,142.860598,40829.353167,7.271543,NaN,386.803057,16.823773,52.297113,4.957420,0


In [28]:
ans = df['ph'].dropna(axis=0)
result = df[df['ph'] <= df['ph'].quantile(0.25)]
print(result['ph'].mean())

5.057093462441731


In [54]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")
train.head()

,age,sex,bmi,children,smoker,region,charges
0,64,female,39.330,0,no,northeast,14901.51670
1,47,female,27.830,0,yes,southeast,23065.42070
2,52,female,33.300,2,no,southwest,10806.83900
3,33,female,22.135,1,no,northeast,5354.07465
4,30,male,31.400,1,no,southwest,3659.34600


In [72]:
target1 = train[train['smoker'] == 'no'].sort_values('charges', ascending=False).reset_index()
target2 = train[train['smoker'] == 'yes'].sort_values('charges', ascending=False).reset_index()
ans1 = target1.loc[:len(target1)*0.1 - 1].charges.mean()
ans2 = target2.loc[:len(target2)*0.1 -1 ].charges.mean()
print(ans2 - ans1)

29297.954548156158


In [73]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,8961990160,20150413T000000,567500.0,3,2.5,2080,4556,2.0,0,0,...,8,2080,0,1999,0,98074,47.6036,-122.014,1530,5606
1,9455200205,20140604T000000,525000.0,3,2.0,1540,7800,1.0,0,0,...,8,1540,0,2004,0,98125,47.7041,-122.288,1510,7800
2,7853220670,20140918T000000,540000.0,3,2.5,2860,8935,2.0,0,0,...,8,2860,0,2004,0,98065,47.5336,-121.855,2650,6167
3,3298201170,20141110T000000,350000.0,3,1.0,940,7811,1.0,0,0,...,6,940,0,1959,0,98008,47.6195,-122.118,1180,7490
4,7972604355,20140521T000000,218000.0,3,1.0,1020,7874,1.0,0,0,...,7,1020,0,1956,0,98106,47.5175,-122.346,1290,7320


In [86]:
target = df[df['bedrooms'] == df['bedrooms'].value_counts().index[0]]
ans = target['price'].quantile(0.9) - target['price'].quantile(0.1)
print(ans)

505500.0


In [87]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,246,328,110,4,4.0,2.5,9.02,1,0.81
1,100,323,113,3,4.0,4.0,8.88,1,0.79
2,79,296,95,2,3.0,2.0,7.54,1,0.44
3,53,334,116,4,4.0,3.0,8.00,1,0.78
4,444,321,114,5,4.5,4.5,9.16,1,0.87


In [105]:
from sklearn.ensemble import RandomForestRegressor

x = df.drop(['Serial No.', 'Chance of Admit'],axis=1)
y = df['Chance of Admit']

rf = RandomForestRegressor(random_state=810)
rf.fit(x, y)
result = pd.DataFrame({'importance': rf.feature_importances_},x.columns).sort_values('importance',ascending=False)
print(result)

                   importance
CGPA                 0.812612
GRE Score            0.083281
TOEFL Score          0.033737
LOR                  0.025138
SOP                  0.024415
University Rating    0.013494
Research             0.007324


In [114]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")
df.head()
df.quality.value_counts()

,count
quality,
5,538
6,516
7,162
4,39
8,15
3,9


In [128]:
target = df[(df['quality'] == 3) | (df['quality'] ==8)]
ans = (target[target['quality']==8].std() - target[target['quality']==3].std()).sort_values(ascending=False).index[0]
print(ans)

total sulfur dioxide


In [130]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,20,F,NORMAL,NORMAL,9.281,4
1,24,M,HIGH,NORMAL,9.475,1
2,34,M,NORMAL,HIGH,22.456,0
3,17,M,NORMAL,NORMAL,10.832,4
4,72,M,LOW,HIGH,16.310,0


In [140]:
df['aa'] = df['Age'] // 10 *10
target = df[df['Sex']=='M']
ans = target.groupby('aa')['Na_to_K'].mean().to_frame()
print(ans)

      Na_to_K
aa           
10  13.627000
20  18.260769
30  15.244143
40  13.897273
50  14.811273
60  14.284308
70  11.363857


In [141]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
df.head()

,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,2.72,14,4.53,0.6,2.718,87.93,0.6,52.758,92.46,5.0,...,0.4,0,0.2,0.0,4.0,108.362,0.4,0.5,21.6724,1
1,3.41,16,12.68,0.6,7.608,41.00,0.6,24.600,53.68,5.5,...,0.4,0,0.2,0.0,4.4,41.936,0.4,0.5,8.3872,1
2,2.72,11,3.11,0.6,1.866,113.97,0.6,68.382,117.08,5.5,...,0.4,0,0.2,0.0,4.4,88.832,0.4,0.5,17.7664,1
3,3.41,8,1.12,0.4,0.448,0.00,0.2,0.000,1.12,5.0,...,0.4,0,0.2,0.0,2.2,1.848,0.4,0.5,0.3696,0
4,55.57,9,1.06,0.4,0.424,0.00,0.2,0.000,1.06,5.0,...,0.4,0,0.2,0.0,2.2,1.824,0.4,0.5,0.3648,0


In [144]:
target = df[['Risk', 'Score_A', 'Score_B']]
ans = target.groupby('Risk').mean()
print(ans)

       Score_A   Score_B
Risk                    
0     0.262234  0.225532
1     0.490164  0.444262


In [145]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")
df.head()

,motion_0,motion_1,motion_2,motion_3,motion_4,motion_5,motion_6,motion_7,motion_8,motion_9,...,motion_55,motion_56,motion_57,motion_58,motion_59,motion_60,motion_61,motion_62,motion_63,pose
0,-6.0,8.0,5.0,6.0,-12.0,-27.0,5.0,24.0,-8.0,-5.0,...,-1.0,-3.0,-6.0,-9.0,-7.0,36.0,68.0,3.0,-3.0,1
1,-32.0,-3.0,-4.0,-5.0,-1.0,16.0,0.0,-5.0,42.0,2.0,...,-17.0,-12.0,-1.0,-1.0,0.0,-26.0,-11.0,-4.0,-9.0,1
2,35.0,-2.0,-1.0,1.0,-25.0,-16.0,-3.0,-20.0,10.0,1.0,...,6.0,-65.0,6.0,-10.0,-31.0,-19.0,-3.0,-47.0,-17.0,0
3,-15.0,-6.0,-3.0,-5.0,27.0,37.0,-2.0,-2.0,19.0,5.0,...,2.0,11.0,0.0,-2.0,-4.0,-7.0,15.0,3.0,14.0,1
4,60.0,3.0,0.0,-7.0,-6.0,-10.0,-6.0,7.0,-11.0,-4.0,...,-12.0,17.0,-3.0,1.0,2.0,-2.0,20.0,18.0,10.0,0


In [1]:
target = df.groupby('pose').median().T

ans = abs(target[0] - target[1]).sort_values(ascending=False).reset_index()

print(ans[ans[0] == ans[0].max()]['index'].values)

NameError: name 'df' is not defined

In [2]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")
df.head()

,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,Tucson,2016,12795,Manual,36263,Diesel,30,61.7,1.7
1,I10,2012,3995,Manual,22089,Petrol,20,61.4,1.2
2,I30,2019,24000,Manual,555,Petrol,145,34.0,2.0
3,Tucson,2017,12995,Manual,32147,Diesel,30,61.7,1.7
4,Tucson,2018,14991,Semi-Auto,32217,Diesel,150,57.6,1.7


In [8]:
target = df.value_counts('model').index[0:3]
print(target)
ans = df[df['model'].isin(target)].groupby('model').price.mean()
print(ans)

Index([' Tucson', ' I10', ' I30'], dtype='object', name='model')
model
I10        7646.137891
I30       11651.821759
Tucson    15805.441373
Name: price, dtype: float64


In [9]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,3,102,74,0,0,29.5,0.121,32,0
1,2,144,58,33,135,31.6,0.422,25,1
2,5,136,82,0,0,0.0,0.640,69,0
3,13,145,82,19,110,22.2,0.245,57,0
4,1,117,60,23,106,33.8,0.466,27,0


In [13]:
ans = df.groupby('Outcome').mean().diff().iloc[1,]
print(ans)

Pregnancies                  1.574159
Glucose                     29.943995
BloodPressure                3.202079
SkinThickness                2.952033
Insulin                     33.398645
BMI                          4.541437
DiabetesPedigreeFunction     0.143646
Age                          5.866939
Name: 1, dtype: float64


In [14]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000
1,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
2,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
3,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
4,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200


In [34]:
pd.set_option('display.max_columns',50)
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,0.8,2.1,0.359,2.9,6.1,0.468,0.486,1.1,1.8,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
1,2,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,0.0,0.0,0.000,2.8,5.0,0.548,0.547,1.4,2.6,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
2,3,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,0.0,0.1,0.000,7.3,12.9,0.562,0.557,4.6,6.1,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1
3,4,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,0.2,1.5,0.125,1.5,2.6,0.560,0.424,0.6,1.0,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1
4,5,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,0.3,1.0,0.304,5.1,8.8,0.578,0.566,1.9,2.2,0.873,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9


In [38]:
ans = df[df['Tm'] == 'TOR'].Age.mean()
print(round(ans,4))

24.8696


In [42]:
ans = df[df['Age'] == df['Age'].min()].Pos.value_counts()
print(ans.index[0])

SG


In [50]:
ans = df['Player'].str.split(' ').str[0].str.lower().value_counts()
print(ans.head(1))

Player
justin    14
Name: count, dtype: int64


In [62]:
ans = df.groupby('Pos')['PTS'].mean().sort_values()
print(ans)

Pos
SF        7.270253
SG-SF     7.360000
C         7.690769
PF        7.737500
SG        7.810553
PG        8.325161
PF-SF     8.500000
SF-SG     8.660000
SG-PG     9.525000
C-PF     12.850000
PG-SG    22.000000
Name: PTS, dtype: float64


In [84]:
ans = df.sort_values(['Tm','G']).groupby('Tm').tail(1).G.mean()
print(ans)

77.51612903225806


In [106]:
ans = df.sort_values(['Tm','G'], ascending=False).groupby('Tm').head(1).G.mean()
print(ans)

77.51612903225806


In [110]:
ans = df[(df['Tm']== 'MIA') & (df['Pos'].isin(['C','PF']))].MP.mean()
print(ans)


16.7875


In [114]:
ans = df[(df['G']) >= (df['G'].mean()*1.5)]['3P'].mean()
print(ans)

1.3853658536585365


In [117]:
ans = df[df['Age'] >= df['Age'].mean()].G.mean() - df[df['Age'] <= df['Age'].mean()].G.mean()
print(ans)

3.787674551781862


In [121]:
ans = df.groupby('Tm').Age.mean().sort_values()
print(ans.index[0])

MEM


In [124]:
ans = df.groupby('Pos').MP.mean()
print(ans)

Pos
C        16.990000
C-PF     25.350000
PF       17.937500
PF-SF    27.300000
PG       19.547742
PG-SG    37.200000
SF       17.514557
SF-SG    20.340000
SG       18.554271
SG-PG    22.950000
SG-SF    19.620000
Name: MP, dtype: float64
